In [6]:
nbIndividus = 100 # nombre d'individus dans le graphe

# taille des clusters du graphe (min/max)
foyer = (2, 5)
communaute = (4, 5) # nb de foyers dans une communauté

# application
joursNotif = 14

# probas initialisation
rencontreFoyer = 0.9 # proba de rencontrer un autre membre du foyer
rencontreCommunaute = 0.01 # proba de rencontre un autre membre de la communauté
rencontreNational = 0.001

# probas


In [ ]:
SAIN=0
MALADEASYMP=1
MALADE=2
DECEDE=3

In [7]:
# Initialisation du graphe
individus = [{"state": SAIN} for individu in range(nbIndividus)]
foyers = []
communautes = []

adj = [[] for l in range(nbIndividus)]
adj[0].append({"noeud": 3, "proba": 0.2})

In [8]:
rencontres = [[[] for jour in range(joursNotif)] for individu in range(nbIndividus)]

In [11]:
# Passage au jour suivant du graphe
def step(graphe):
    for i in range(nbIndividus):
        pass

In [5]:
# Affichage
